본 글은 Bert Carremans 님의 https://www.kaggle.com/bertcarremans/data-preparation-exploration 을 참조하였음을 밝힙니다. 좋은 커널 공유에 감사드립니다.

This page is originated from the masterwork of Bert Carremans (https://www.kaggle.com/bertcarremans/data-preparation-exploration). I'm honored to step on the giant's shoulder. I appreciate you, indeed.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.preprocessing import Imputer  # imputer 더 이상 지원안함.
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

### Table of Contents
1. [데이터의 전체적인 특징](#chapter1)
2. [메타데이터프레임 만들기](#chapter2)
3. [통계 분석](#chapter3)
4. [데이터 전처리](#chapter4)
5. [EDV(Exploratory Data Visualization)](#chapter5)
6. [Feature Enginering](#chapter6)
7. [Feature 고르기](#chapter7)
8. [Feature 스케일링](#chapter8)

In [ ]:
train_df = pd.read_csv("../input/porto-seguro-safe-driver-prediction/train.csv")
train_df.head(5)

In [ ]:
test_df = pd.read_csv("../input/porto-seguro-safe-driver-prediction/test.csv")
test_df.head()

### 해당 데이터의 전체적 특징 <a class="anchor" id="chapter1"></a>
대회를 수행하기 위해서 필요한 데이터 상세 내용의 전체 요약입니다.
- 비슷한 그룹핑에 들어있는 피쳐는 다음과 같이 표시되어 있음. (ind, reg, car, calc ...)
- bin 이라고 되어 있는 칼럼은 binary, cat은 고양이가 아니라 카테고리형 데이터이다.
- 아무런 표시가 없는 데이터 칼럼은 연속적인 데이터이다.
- 결측치는 NaN이 아니라 -1로 표시되어 있다.
- Target 칼럼은 0이면 내년도에 보험에 들지 않았고, 1이면 보험을 들었다. 

In [ ]:
train_df.tail()

앞으로 체크해야 할것
- binary 변수들
- 정수값으로 구성되어 있는 카테고리 변수들
- 정수나 실수값으로 구성되어 있는 변수들
- -1로 되어 있는 결측치
- 타겟 변수와 id 변수

train_df 살펴보자

In [ ]:
train_df.info()

595212개의 행이 존재하고, 59개의 칼럼이 존재한다. 그중 실수 칼럼은 10개, 정수 칼럼은 49개이다.

테스트 데이터도 확인해보자.

In [ ]:
test_df.info()

892816개의 행, 58개의 열로 구성되어 있다. test데이터셋은 target이 존재하지 않으므로 문제가 없어보인다.

### Metadata <a class="anchor" id="chapter2"></a>
효과적으로 데이터프레임을 분석하기 위해서, 우리는 변수들에 관한 메타 정보들을 새롭게 저장할 것이다. 메타데이터는 우리가 상세한 분석이나 시각화, 모델링 등에 활용될 수 있다.
아래는 데이터의 특성에 따라 4가지 항목으로 분류한 것이다. 영어로 적는게 더 편할 것 같아 따로 한글로 바꾸진 않았다,
* role : input, ID, target
* level : nominal, interval, ordinal, binary
* keep : True or False
* dtype : int, float, str

In [ ]:
train_df.columns

아래는 metadata를 분류하는 작업이다. 

In [ ]:
data = []
for f in train_df.columns :
    # role 정의하기
    if f == "target" :
        role = "target"
    elif f == "id" :
        role = "id"
    else :
        role = "input"
        
    # level 정의하기
    if ("bin" in f) or (f == "target") :
        level = "binary"
    elif ("cat" in f) or f =="id" :
        level = "nominal"
    elif train_df[f].dtype == float :
        level = "interval"
    elif train_df[f].dtype == int :
        level = "ordinal"
        
    # id값만 제외하고 모든 값들에 keep 을 True로 설정
    keep = True
    if f == "id" :
        keep = False
    
    # 데이터타입을 정의
    dtype = train_df[f].dtype
    
    # 위에서 구한 메타데이터를 하나의 딕셔너리로 담는다.
    f_dict = {
        "varname": f,
        "role" : role,
        "level" : level,
        "keep" : keep,
        "dtype" : dtype
    }
    data.append(f_dict)


메타데이터를 데이터프레임에 저장한다.

In [ ]:
meta = pd.DataFrame(data, columns=["varname", "role", "level", "keep", "dtype"])
meta.set_index("varname", inplace=True)
meta

예시로, 숫자 데이터가 포함되어 있고 dropped 되지 않은 열을 불러온다.

In [ ]:
meta[(meta.level == "nominal") & (meta.keep)].index

아래는 role과 level이 총 몇개가 들어있는지에 대한 데이터프레임이다.

In [ ]:
pd.DataFrame({"count" : meta.groupby(["role", "level"])["role"].count()})

In [ ]:
pd.DataFrame({"count" : meta.groupby(["role", "level"])["role"].count()}).reset_index()

## 통계 분석 <a class="anchor" id="chapter3"></a>

보통 통계 분석을 할때는 describe()함수를 사용하지만, mean, std, 등을 카테고리나 id 값에 사용하는 것은 적절하지 않다. 우리는 그 대신 아까 구한 메타 데이터를 이용한다. 메타 데이터 덕분에 우리는 describe()를 사용하고 싶은 칼럼만 더 쉽게 구할 수 있다.

### 구간 값 변수

In [ ]:
v = meta[ (meta.level == "interval") & (meta.keep)].index
v
# 구간값이 있는 칼럼의 이름만 구했다.

In [ ]:
# 구한 칼럼들을 describe()함수에 넣는다.
train_df[v].describe()

* min을 잘 살펴보자.
* min이 -1이 있는 칼럼을 나열한다.


### reg 변수
- ps_reg_03가 결측치 포함
- 각 변수들의 최소-최대값의 범위는 굉장히 다양하다. 정규화를 사용할 수 있지만, 정규화는 classifier에 달려있다.

### car 변수
- ps_car_12, ps_car_14가 결측치를 포함한다.
- 스케일링 사용할것임

### calc 변수
- 결측치 없음.
- 세 calc 최대값이 모두 0.9이다. 
- 세 calc 변수들은 비슷한 분포를 가지고 있다.

전체적으로, 범위 변수들의 범위가 상당히 작다. 아마도 로그화나 다른 데이터 변형 작업이 실행되었을 가능성이 있다 (개인정보 보호를 위해?)


### 순열 변수들

In [ ]:
v = meta[ (meta.level == "ordinal") & (meta.keep) ].index
train_df[v].describe()

- ps_car_11에 결측치 포함
- 이 값에 스케일링을 넣으면 표준화 가능

### 이진 변수들

In [ ]:
v = meta[ (meta.level == "binary") & (meta.keep)].index
train_df[v].describe()

이진변수이므로 모두 0,1만 들어있다.
- target이 포함되어있는 이진 변수들의 평균은 3.645%로, 매우 불균형적으로 구성되어 있다.
- 낮은 평균을 고려해보면 대부분의 경우 대부분의 타겟값들은 0이라고 할 수 있다.

---

### 불균형 클래스 조정하기 <a class="anchor" id="chapter4"></a>
위에서 설명했듯, 타겟 칼럼이 1인 비율은 0인 비율에 비해 훨씬 낮은 것을 알 수 있다. 이 결론을 바탕으로 모델을 만들때 꽤 정확하게 예측하는 모델을 만들 수는 있지만, 다른 추가된 값을 실질적으로 포함할 수 있다. 이것을 바탕으로, 2개의 방법을 고려해볼수 있겠다.
- target이 1인 항목을 oversampling 하는것.
- target이 0인 항목을 undersampling 하는것.

(그 외에도 다른 여러 방법들이 있다. [MachineLearningMastery.com](https://www.kaggleusercontent.com/kf/1676042/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..BpxnVrgBAuCqyykqSJ6rRg.sG9fZig1ih6alTOjfM8BU0n0yWwYUrDMWnn7gAWRUTIPvMBMYoj5yQY1vnnrGEvz5AzBC7BcY5WHrfrYinI9dKs2LGrCtKHzzJOxHn5yjaCIyQOTduufatzf-i-s4az0TBmjzwD_PsCDJ6bLYDB0pl3vc60cq3n_ffCmSnOiqwfGYJbk3dDGN-rSX38ZLPDfPxyIkMAHM2SAzzINip-uDk1PaxzzjkZ5ORN9WnirQOFONaKRnin9SRxJteTfl3wBdC9-1owwSrsSNJrlTUjGMP-bENnhuiZnUfURh5aibHmwm8oaccaREQriYQP3M4ldrdAAwORXoAmVVW1V2KWfpEfe6M0L4DqlXi1X3qcynHj8WBBlxqnLyhAL-PF0EGZeqB_jPkudXq0ZXGdHPbqwsPS_ZkbcUVSVdNhSx-NmwKL_bjlYHUAwrSU0kUdoDMYhye_xj6z4e5rqzxfTgXiiABdXYPCNOXX2lqTzQnnlHOLDhCM4SxbzGzdPklVi1L-LwAzVTz9kYDGzwNuomCn9bcGz6DZkNOVeMOfwzSpQuf8qMDM1-yjbZXdj-iDTDDApybgt8M-ukCLE5Z48K7SPY_rmWdWV75ads1RvRNuJY_i_OeciAZ0GyiruGfrMvXMQ0L2LsW6fREg2-WsIhk11Oc53-xyeOSmH6jl-8KRunbk.ZY90E-09ljdpHL8dqGb6_A/(https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)
)
- 현재 트레이닝 모델에 있는 데이터 셋이 많으므로, target이 0인 항목을 undersampling을 하는것으로 결정한다.

In [ ]:
desired_apriori = 0.10

# 타겟의 인덱스 구하기
idx_0 = train_df.query("target == 0").index
idx_1 = train_df.query("target == 1").index

idx_0


In [ ]:
idx_1

target이 0인 행이 573518개인 반면, target이 1인 행은 21694개 밖에 되지 않는다. 

In [ ]:
# 각 행의 갯수 저장
nb_0 = len(idx_0)
nb_1 = len(idx_1)

In [ ]:
# undersampling을 할 비율
undersample_rate = ((1-desired_apriori) *nb_1) / (nb_0 * desired_apriori)
undersampled_nb_0 = int(undersample_rate * nb_0)
print(f"언더샘플을 할 비율 : {undersample_rate}")
print(f"언더샘플 후에 target 0의 개수 : {undersampled_nb_0}")


In [ ]:
# 목표 priori에 도달하기 위해 언더샘플된 랜덤으로 target=0 행의 인덱스의 순서를 섞는다.
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# 두 인덱스들을 합쳐 하나의 리스트로 저장
idx_list = list(undersampled_idx) + list(idx_1) 

# undersample된 데이터프레임을 새롭게 저장한다.
train_df = train_df.loc[idx_list].reset_index(drop=True)
train_df

In [ ]:
train_df["target"].value_counts()

# 데이터 전처리 <a class="anchor" id="chapter4"></a>

### 결측치 확인
- 결측치는 -1로 저장되어 있다.

In [ ]:
vars_with_missing = []

for f in train_df.columns :
    missings = train_df[train_df[f] == -1][f].count()
    if missings > 0 :
        vars_with_missing.append(f)
        missings_perc = missings/train_df.shape[0]
        
        print(f"{f}칼럼은 {missings}개의 결측치가 있습니다. {missings_perc:.2%}")
        

print(f"전체 {len(vars_with_missing)}개의 결측치가 있습니다.")

* ps_car_03_cat와 ps_car_05_cat 는 결측치가 각각 69%, 44%로 너무 많다. 칼럼을 삭제하는게 더 좋을 것 같다.
* 다른 카테고리형 칼럼들은 결측치가 매우 미소하므로 결측치 부분만 제거해주도록 하자.
* ps_reg_03 카럼은 총 17%의 결측치가 있다. 이 칼럼은 연속적인 값이므로 평균을 구해서 채워넣도록 하자.
* ps_car_11은 1개의 결측치가 있다. 평균값을 구해서 넣어주도록 하자.
* ps_car_14는 전체의 7퍼센트가 결측치이고 연속값이므로 평균을 채워주도록 하자.

In [ ]:
# ps_car_03_cat과 ps_car_05_cat 칼럼 제거
train_df.drop(["ps_car_03_cat","ps_car_05_cat" ], axis=1, inplace=True)
train_df.head()

In [ ]:
# 메타 데이터프레임 업데이트
meta.loc[["ps_car_03_cat","ps_car_05_cat"], "keep"] = False
meta

In [ ]:
from sklearn.impute import SimpleImputer as Imputer

In [ ]:
# 임퓨팅 하기 0
mean_imp = Imputer(missing_values = -1, strategy="mean")
mode_imp = Imputer(missing_values=-1, strategy="most_frequent")

In [ ]:
# 결측치 처리
train_df["ps_reg_03"] = mean_imp.fit_transform(train_df[["ps_reg_03"]]).ravel()


In [ ]:
train_df["ps_reg_03"]

In [ ]:
train_df["ps_car_14"] = mean_imp.fit_transform(train_df[["ps_car_14"]]).ravel()
train_df["ps_car_12"] = mean_imp.fit_transform(train_df[["ps_car_12"]]).ravel()
train_df["ps_car_11"] = mean_imp.fit_transform(train_df[["ps_car_11"]]).ravel()
train_df

In [ ]:
# 결측치 다시 확인
vars_with_missing = []

for f in train_df.columns :
    missings = train_df[train_df[f] == -1][f].count()
    if missings > 0 :
        vars_with_missing.append(f)
        missings_perc = missings/train_df.shape[0]
        
        print(f"{f}칼럼은 {missings}개의 결측치가 있습니다. {missings_perc:.2%}")
        

print(f"전체 {len(vars_with_missing)}개의 결측치가 있습니다.")

이제 categorical 데이터를 정제해준다.

### 카테고리형 변수 내의 cardinality(카테고리의 종류) 체크하기
cardinality는 한 카테고리 내에 있는 다른 종류의 값의 개수입니다. 현재는 없지만 나중에 우리가 가짜 더미 변수들을 카테고리변수에서 가져와 만들건데, 그래서 카테고리 변수 내의 인자값이 얼마나 있는지를 알아야합니다. 각 변수들을 각각 처리해야하지 않으면 나중에 더미 데이터를 만들때 문제가 있을 수도 있습니다.

In [ ]:
meta

In [ ]:
# 메타 데이터프레임에서 카테고리형 데이터만 가져옴
v = meta[(meta.level == "nominal") & (meta.keep)].index
v

In [ ]:
for f in v : 
    # cardinality 구하기
    dist_values = train_df[f].value_counts().shape[0]
    print(f"변수 {f}는 총 {dist_values}개의 종류로 되어 있습니다.")

ps_car_11_cat 변수가 104개의 값을 가지고 있는 것을 확인했습니다. 이제 이 타겟을 frequency encoding을 이용해서 인코딩 해줍니다.

In [ ]:
# add_noise, 인코딩 함수 선언.


# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features

def add_noise(series, noise_level) :
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(train_series = None, test_series= None, target = None, 
                 min_samples_leaf = 1, smoothing = 1, noise_level=0) :
    '''
    Smoothing 과정은 Daniele Micchi-Barreca님의 글을 참조해서 만들었습니다.
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    
    train_series : pd.Series로 저장된 train의 카테고리형 칼럼
    test_series : pd.Series로 저장된 test의 카테고리형 칼럼
    target : pd.Series로 저장된 target 값
    min_samplies_leaf (int) : 카테고리 평균을 고려한 최소 샘플 개수
    smoothing (int) : 카테고리 평균과 prior 을 밸런싱하는 smoothing effect
    '''
    # asserting해준다. 
    assert len(train_series) == len(target)
    assert train_series.name == test_series.name
    temp = pd.concat([train_series, target], axis=1)
    
    # target 평균 계산하기
    averages = temp.groupby(by=train_series.name)[target.name].agg(["mean", "count"])
    
    # smoothing 구하기
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing) )
    
    # average를 모든 타겟 데이타에 적용합니다.
    prior = target.mean()
    # count 값이 클 수록, full_avg는 적게 영향을 미칩니다.
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # 구한 averages를 통해 train_series와 test_series에 적용합니다.
    ft_train_series = pd.merge(train_series.to_frame(train_series.name),
                              averages.reset_index().rename(columns={"index" : target.name, 
                                                                    target.name : "average"}),
                              on=train_series.name,
                              how = "left")["average"].rename(train_series.name + "_mean").fillna(prior)
    # pd.merge는 인덱스를 저장하지 않으므로 인덱스를 다시 설정해주자.
    ft_train_series.index = train_series.index
    # fit test_series
    ft_test_series = pd.merge(test_series.to_frame(test_series.name),
                             averages.reset_index().rename(columns={"index" : target.name,
                                                                   target.name : "average"}),
                             on=test_series.name, 
                              how="left")["average"].rename(train_series.name + "_mean").fillna(prior)
    # pd.merge는 인덱스를 저장하지 않으므로 인덱스를 다시 설정해주자.
    ft_test_series.index = test_series.index
    
    return add_noise(ft_train_series, noise_level), add_noise(ft_test_series, noise_level)

    

In [ ]:
# 인코딩
train_encoded, test_encoded = target_encode(train_df["ps_car_11_cat"],
                                           test_df["ps_car_11_cat"],
                                           target=train_df.target,
                                           min_samples_leaf=100,
                                           smoothing=10,
                                           noise_level=0.01)

# 인코딩 값 저장
train_df["ps_car_11_cat_te"] = train_encoded
train_df.drop("ps_car_11_cat", inplace=True, axis=1)
meta.loc["ps_car_11_cat", "keep"] = False  # meta 데이터프레임 업데이트
test_df["ps_car_11_cat_te"] = test_encoded
test_df.drop('ps_car_11_cat', axis=1, inplace=True)

In [ ]:
# 확인작업
train_df["ps_car_11_cat_te"]

카테고리 데이터가 잘 인코딩 되었다.

## EDV(Exploratory Data Visualization)  <a class="anchor" id="chapter5"></a>
### 데이터 시각화 하기

### 카테고리형 변수
카테고리형 변수들과 타겟이 1인 고객의 비율을 알아보자.

In [ ]:
# 카테고리형 변수들 이름 가져오기
v = meta[(meta.level == "nominal") & (meta.keep)].index
v

In [ ]:
for f in v : 
    plt.figure()
    fig, ax = plt.subplots(figsize = (20, 10))
    
    # category형 값 당 존재하는 타겟이 1인 항목의 비율 계산
    cat_perc = train_df[[f, "target"]].groupby([f], as_index=False).mean()
#     print(cat_perc)
    # value 값으로 정렬
    cat_perc.sort_values(by="target", ascending=False, inplace=True)
#     print(cat_perc)
    
    # 막대그래프 그리기
    sns.barplot(ax = ax, x = f, y="target", data=cat_perc, order=cat_perc[f])
    plt.ylabel("% target", fontsize=18)
    plt.xlabel(f, fontsize=18)
    plt.tick_params(axis="both", which="major", labelsize=18)
    plt.title(f, fontsize=18)
    plt.show()

위에서 본 것과 같이 결측치가 많이 존재하는 변수들이 있다. 그렇기 떄문에 많은 결측치들을 별도의 카테고리 값으로 만드는것이 mode로 replace하는 것보다 더 바람직하다. 결측치를 가지고 있는 고객들은 더 많이 (물론 100%는 아니지만) 보험을 드는 경향이 있다. 

### 범위 변수들
이제는 범위값을 가지고 있는 변수들의 상관관계를 확인해보도록 하자. 상관관계를 알아볼 때 가장 사용하기 좋은 것은 히트맵으로 시각화 하는 방법이다. [Michael Waskom 님의 글을 참조하였다.](http://seaborn.pydata.org/examples/many_pairwise_correlations.html)

In [ ]:
# 히트맵 함수를 정의한다.
def corr_heatmap(v) :
    correlations = train_df[v].corr()
    
    # 2개의 색을 가진 컬러맵 생성
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt=".2f", square=True, 
                linewidths=0.5, annot=True, cbar_kws={"shrink": 0.75})
    plt.show()

In [ ]:
# meta에서 interval인것을 추출
v = meta[ (meta.level == "interval") & meta.keep].index
corr_heatmap(v)

- ps_reg_02와 ps_reg_03은 상관계수가 0.7로 높은 관계가 있다.
- ps_car_12와 ps_car_13은 상관계수가 0.67
- ps_car_12와 ps_car_14는 상관계수가 0.58
- ps_car_13와 ps_car_15는 상관계수가 0.53이다.

Seaborn은 시각화하게 편리한 도구가 많아 변수들간의 관계(특히 선형관계)를 잘 확인할 수 있다. 그 중 하나인 pairplot을 사용하면 한눈에 변수들간의 관계를 확인할 수 있지만, 이미 우리가 히트맵에서 확인했듯이, 상관계수가 높은 카테고리를 이미 확인하였다. 그래서 우리는 바로 각각의 높은 상관계수를 가지고 있는 변수를 살펴보는게 좋겠다.

##### NOTE : 데이터 분석 과정 속도를 내기 위해서 train_data의 샘플을 사용하였다.

In [ ]:
# 데이터의 10%를 랜덤하게 추출
s = train_df.sample(frac=0.1)
s

#### ps_reg_02와 ps_reg_03의 상관관계 확인
아래의 회귀선에서 보이듯, 두 변수들에 1차 함수 형태의 비례 관계가 있음을 알 수 있다. hue 파라미터 덕분에 우리는 target=0의 회귀선과 target=1의 회귀선이 같음을 알 수 있다.

In [ ]:
sns.lmplot(x="ps_reg_02", y="ps_reg_03", data=s, hue="target", 
           palette="Set1", scatter_kws={"alpha" : 0.3})
plt.show()

ps_car_12와 ps_car_13의 관계

In [ ]:
sns.lmplot(x="ps_car_12", y="ps_car_13", data=s, hue="target",
          palette="Set1", scatter_kws={"alpha" : 0.3})
plt.show()

0.4 ~ 0.6 사이에 값이 집중되어 있고 회귀선이 같음을 알 수 있다.

ps_car_12와 ps_car_14의 관계

In [ ]:
sns.lmplot(x="ps_car_12", y="ps_car_14", data=s, hue="target", palette="Set1",
          scatter_kws={"alpha" : 0.3})
plt.show()

ps_car_13과 ps_car_15의 관계

In [ ]:
sns.lmplot(x="ps_car_15", y="ps_car_13", data=s, hue="target", palette="Set1",
          scatter_kws={"alpha" :0.3})
plt.show()

자, 이제 상관관계도 살펴보았다. 그러면 이제 상관관계가 있는 변수 중 어떤 것을 선택해야 할까? PCA(Principal Component Analysis)에 따르면 필요없는 변수들은 제거하여 차원을 줄여야 한다고 한다. 본래는 제거를 해야 마땅하지만 상관계수가 있는 항목이 많지 않으므로 모델에서 heavy-lifting을 사용하도록 하자.

### 순서형 데이터의 상관계수 확인하기

In [ ]:
# 메타에서 ordinal인 것을 가져온다.
v = meta[(meta.level == "ordinal") & (meta.keep)].index
# 히트맵 
corr_heatmap(v)

상관계수가 있는 변수들이 많아 보이지 않는다. 그렇다면, 나중에 target값을 기준으로 grouping 했을 때 분포가 어떤지 살펴보면 되겠다.

## Feature Enginering <a class="anchor" id="chapter6"></a>

#### Dummy 값 생성하기
카테고리의 값은 그저 다른 상태라는 것만을 알려주지, 서로간의 어떤 순서나 배수 관계가 아니다. 예를 들자면, 카테고리 값이 2라고 해서 그게 카테고리 값 1인 값의 2배가 아니라는 것이다. 그렇기 때문에, 이런 관계를 이용해서 우리는 더미 값을 만들 수 있다. 처음 생성된 더미 변수를 버린다. 왜냐하면 첫 더미 변수들은 순수한 변수들의 카테고리 값에서 만들어진 다른 더미 변수들에서 오기 떄문이다.

In [ ]:
v = meta[(meta.level == "nominal") & (meta.keep)].index
print(f"더미 만들기 작업 전 변수 개수 : {train_df.shape[1]} ")
train_df = pd.get_dummies(train_df, columns=v, drop_first=True)
print(f"더미 만들기 작업 후 변수 개수 : {train_df.shape[1]} ")


52개의 더미 칼럼이 추가 되었다!

### interaction(범위) 변수 생성하기

In [ ]:
# level이 interval인 칼럼 가져오기
v = meta[ (meta.level == "interval") & (meta.keep)].index

# 차수가 2개인 Polynomial 값 추가하기
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(train_df[v]), columns=poly.get_feature_names(v))
interactions

In [ ]:
# 기존의 칼럼 제거하기
interactions.drop(v, inplace=True, axis=1)
print(f"interactions 생성 전 칼럼 수 : {train_df.shape[1]}")
train_df = pd.concat([train_df, interactions], axis=1)
print(f"interactions 생성 전 칼럼 수 : {train_df.shape[1]}")

추가 interaction 칼럼이 train 데이터에 추가되었다. get_feature_names 메소드 덕분에 우리는 칼럼 이름들을 새로운 변수에 집어넣을 수 있다.

### Feature 고르기 <a class="anchor" id="chapter7"></a>

#### varience가 낮거나 0인 변수 제거하기

개인적으로, 나는 classifier 알고리즘이 알아서 어떤 피쳐값을 유지할 건지 선택하도록 하는걸 더 좋아하는데, 우리 스스로 할 수 있는게 하나 있다. 그건 바로 variance가 낮거나 0인 변수를 제거하는것이다. Sklearn은 VarianceThreshold 간편한 메소드를 가지고 있다. 기본적으로 VarianceThreshold 메소드는 variance가 0인 변수들을 지운다. 근데 이번 대회에서는 이것을 사용하지 않을 것이다 왜냐하면 앞 과정에서 variance가 0인 변수들을 찾을 수 없었기 때문이다. 근데 1% 미만인 variance를 삭제한다면 총 31개의 칼럼을 삭제하게 된다.

In [ ]:
# VarienceThreshold 생성자 선언
selector = VarianceThreshold(threshold=0.01)
# id와 target 값 없이 학습 실행
selector.fit(train_df.drop(["id", "target"], axis=1))

In [ ]:
# x를 not x로 만들어주는 벡터 
f = np.vectorize(lambda x : not x)

v= train_df.drop(["id", "target"], axis=1).columns[f(selector.get_support())]
print(f"{len(v)}개의 칼럼의 variance가 매우 낮다.")
print(f"variance가 낮은 칼럼 :{list(v)}")

만약 variance로 변수를 선택한다면 꽤 많은 변수를 잃어야 한다. 하지만 현재 데이터 셋은 변수가 그렇게 많지 않으므로 classifier가 대신 선택하게 하자. 많은 변수가 있는 데이터 셋에는 classifier 만큼 시간을 절약할 수 있는게 없다. 


Sklearn은 또한 다른 feature selection 메소드를 활용할 수 있는 툴이다. 그 중 하나인 SelectFromModel 메소드는 분류기로 하여금 최고의 피쳐를 자동으로 골라 그것들을 사용하게 한다. 아래서 SelectFromModel 메소드를 RandomForest와 함께 사용하는 방법을 소개한다 

### 랜덤포레스트와 SelectFromModel을 이용해서 피쳐 선택하기

여기서 우리는 피쳐 선택을 랜덤포레스트의 feature importances를 이용해 할것이다. Sklearn의 SelectFromModel을 이용하면 얼마나 많은 변수들을 저장할 것인지 분류할 수 있게 된다. 피쳐 중요도를 수동으로 설정해서 한계를 정할 수 있다. 여기서는 간단하게 상위 50%만 고르도록 하자.
    아래의 코드는 Sebastian Raschka님의 [깃허브 Repository](https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch04/ch04.ipynb)에서 가져온 것이다. 파이썬 머신러닝의 샘플 코드가 예술이다.

In [ ]:
X_train = train_df.drop(["id", "target"], axis=1)
y_train = train_df["target"]

feat_labels = X_train.columns
feat_labels

In [ ]:
# 랜덤포레스트 객체 생성
rf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
importances = rf.feature_importances_

In [ ]:
importances

In [ ]:

indices = np.argsort(rf.feature_importances_)[::-1]
indices

In [ ]:
for f in range(X_train.shape[1]) :
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))

SelectFromModel이 있으므로 우리는 어떤 종류의 분류기를 선택할것인지 정할 수 있고 피쳐 중요도에 대한 한계 구간을 구분할 수 있다. get_support 메소드는 그레사 train_data셋에 있는 변수의 개수를 제한할 수 있다. 

In [ ]:
sfm = SelectFromModel(rf, threshold="median", prefit=True)
print(f"선별 전 피쳐의 개수 : {X_train.shape[1]}")
n_features = sfm.transform(X_train).shape[1]
print(f"선별 후 피쳐의 개수 : {n_features}")
selected_vars = list(feat_labels[sfm.get_support()])

In [ ]:
train_df = train_df[selected_vars + ["target"]]

### 피쳐 scaling <a class="anchor" id="chapter8"></a>

앞서 말했듯이, 우리는 standardScaling을 적용해서 스케일링을 할 수 있다. 몇 분류기들은 scaling을 사용 전과 후가 큰 차이가 나기도 한다.

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(train_df.drop(["target"], axis=1))